In [ ]:
from chathub import chat
from IPython.display import display
import ipywidgets as widgets

chat_area = widgets.Textarea(
    layout={'width': '100%', 'height': '300px'}, disabled=True)
input_box = widgets.Text(layout={'width': '100%'})
display(widgets.VBox([chat_area, input_box]))

manager = chat.Manager(chat.OllamaBot(), chat.GitRepository())
messages = []


def on_submit(change):
    user_input = input_box.value
    input_box.value = ''
    chat_area.value += f"\nUser: {user_input}"
    messages.append({"role": "user", "content": user_input})
    chat_area.value += "\nAssistant: "

    for chunk in manager.mediate("user1", "chat1", messages):
        if content := chunk.get('message', {}).get('content'):
            chat_area.value = chat_area.value + content


input_box.on_submit(on_submit)

/tmp/ipykernel_83386/1297305231.py:26: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_submit)


{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.53395254Z', 'message': {'role': 'assistant', 'content': 'Hey'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.6149242Z', 'message': {'role': 'assistant', 'content': ' there'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.695942506Z', 'message': {'role': 'assistant', 'content': '!'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.777070356Z', 'message': {'role': 'assistant', 'content': ' 👋'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.857369383Z', 'message': {'role': 'assistant', 'content': '\n\n'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abliterated', 'created_at': '2024-11-13T08:11:59.937434771Z', 'message': {'role': 'assistant', 'content': 'How'}, 'done': False}
{'model': 'jaahas/gemma-2-9b-it-abli